# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [ ]:
import pandas as pd 
from sklearn.preprocessing import LabelEncoder, StandardScaler

ticTacToe = pd.read_csv('./tic-tac-toe.csv')
print(ticTacToe.head())
label_encoders = {} 
for column in ticTacToe.columns[:-1]: 
    le = LabelEncoder() 
    ticTacToe[column] = le.fit_transform(ticTacToe[column]) 
    label_encoders[column] = le
print(ticTacToe.head())

X = ticTacToe.drop(columns=['class']) # Assuming 'Result' is the target variable 
ticTacToe['class'] = ticTacToe['class'].astype(int)
y = ticTacToe['class']


#scaler = StandardScaler() 
#X_normalized = scaler.fit_transform(X)

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [ ]:

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow


batch_size = 32
epochs = 50
# Step 1: Split the Training and Test Data
# Assuming X_normalized and y are defined from the previous steps
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Create a Sequential Model
model = Sequential()

# Step 3: Add Layers to the Model
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))

# Step 4: Compile the Model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()





In [ ]:
# Step 5: Fit model data
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(X_test, y_test))

In [ ]:

# Evaluate Model

test_loss, test_acc = model.evaluate(X_test, y_test) 
train_loss, train_acc = model.evaluate(X_train, y_train)
print(f"Train Loss: {train_loss}") 
print(f"Train Accuracy: {train_acc}")
print(f"Test Loss: {test_loss}") 
print(f"Test Accuracy: {test_acc}")


# Step 7: Save the Model
model.save('tic-tac-toe.model.h5')
print("Model saved as tic-tac-toe.model")

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [ ]:
import numpy as np

print(len(X_test))

random_indices = np.random.choice(len(X_test), size=5, replace=False) # Select 5 random indices 
print(random_indices)
random_samples = X_test.iloc[random_indices] 
random_labels = y_test.iloc[random_indices]
# Make predictions
predictions = model.predict(random_samples)
predicted_labels = (predictions > 0.5).astype(int).flatten()

# Display the results
for i, index in enumerate(random_indices):
    print(f"Sample {index}:")
    print(f"Predicted: {predicted_labels[i]}, Actual: {random_labels.iloc[i]}")
    print()


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [ ]:

model2 = Sequential()

# Step 4: Add More Layers to the Model
model2.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model2.add(Dropout(0.3))
model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.3))
model2.add(Dense(32, activation='relu'))
model2.add(Dropout(0.3))
model2.add(Dense(16, activation='relu'))
model2.add(Dense(2, activation='softmax'))  # Use sigmoid for binary classification

# Step 5: Adjust the Learning Rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.003)  # Adjust the learning rate if needed

# Step 6: Compile the Model
model2.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model2.summary()

# Step 7: Fit the Training Data with More Epochs
history = model2.fit(X_train, y_train,
                     batch_size=batch_size,
                     epochs=200,  # Increased the number of epochs
                     verbose=1,
                     validation_data=(X_test, y_test))

# Step 8: Evaluate the Model
test_loss, test_acc = model2.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_acc}")

# Save the Model with .h5 extension
model2.save('tic-tac-toe_model2_improved.keras')
print("Model saved as tic-tac-toe_model2_improved.keras")


**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# 1st it addeda layer then in played with the learinig rate then I added more epoch